<a href="https://colab.research.google.com/github/MohanKumar2002/Chatbot-using-NLTK-and-Tensorflow/blob/master/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model


lemmatizer = WordNetLemmatizer()

intents = json.loads(open('/content/intents.json').read())

words = pickle.load(open('/content/words.pkl', 'rb'))
classes = pickle.load(open('/content/classes.pkl', 'rb'))
model = load_model('/content/chatbotmodel.h5')

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word)  for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words= clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1

    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda  x:x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list


def get_response(intents_list,intents_json):
    tag= intents_list[0]['intent']
    list_of_intents =intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

print(" Welcome to Cube AI Tech Solutions ")
print("Ask your any query about our Company")
while True:
    message = input("| You: ")
    if message == "bye" or message == "Goodbye":
        ints = predict_class(message)
        res = get_response(ints, intents)
        print("| Bot:", res)
        print("The Program End here!")
        exit()
        break

    else:
        ints = predict_class(message)
        res = get_response(ints, intents)
        print("| Bot:", res)

 Welcome to Cube AI Tech Solutions 
Ask your any query about our Company
| You: hi
1/1 [==============================] - 0s 96ms/step
| Bot: What can I do for you?
| You: h
1/1 [==============================] - 0s 31ms/step
| Bot: Please give me more info
| You: where are this located
1/1 [==============================] - 0s 22ms/step
| Bot: Informatics College Pokhara has been in direct partnership with London Metropolitan University, 
UK to provide enviable higher education in IT and Business to students in Pokhara.
For Bachelors Degree in Information Technology we have been offering the specialization in BSc (Hons) Computing.
For Bachelors in Business Administration we have been offering the followings:

1. BBA (Marketing) with International Business 

2. BBA (Accounting & Finance) with International Business

3. BBA (International Business)
| You: bye
1/1 [==============================] - 0s 19ms/step
| Bot: Talk you later
The Program End here!
